In [9]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy import reshape
from scipy import stats
import re
%matplotlib inline

In [10]:

def scrape(i):
    global build_dict_list
    #some entries have been removed at the time of scraping
    try: 
        a = open(i, "r")
        b = BeautifulSoup(a)
        #Parts
        
        #Labels
        labels = b.find('ul', attrs={"class":"parts"}).findAll('p', attrs={"label"})
        
        cols = [label.contents[0] for label in labels]
        count = [0 for x in range(len(cols))]

        #handle multiple parts of the same type (e.g. 2 graphics cards)
        for x in range(len(cols)):
            count[x] = 1+ cols[0:x].count(cols[x])

        new_cols = [str(x)+"_"+str(y) for (x,y) in zip(cols, count)]

        #Links
        link_list = b.find('ul', attrs={"class":"parts"}).findAll('a', attrs={"name"})
        link_list = [link['href'] for link in link_list]

        #Prices
        prices = b.find('ul', attrs={"class":"parts"}).findAll('div', attrs={"price"})
        price_list = [price.contents[0].strip() for price in prices]
        total_price = float(b.find("li",attrs={"class":"partlist-total"}).find('span', attrs={"class":"price"}).contents[0].strip("$"))
        total_price_dict = {"total":total_price}
        
        #dictionary containing Link and Price
        build_part_dict = {part: {"Link":link,"Price":price} for part, link, price in zip(new_cols, link_list, price_list)}
        
        #description
        description_paragraphs = b.find('div', attrs={'class':'description block'}).find_all('p')
        description_clean = ''
        for paragraph in description_paragraphs:
            add = paragraph.text
            description_clean += add + ' '  
        description_dict = {'Description':description_clean}
        

        #Builder: 
        build_name = b.find('h1', attrs={"class":"name"}).contents[0]
        builder_link = b.find('p', attrs={"class":"owner"}).find('a')['href']
        builder_dict = {"Build_Name":build_name}
        owner_dict = {"Builder":builder_link}

        #Details
        detail_vals = [x.strip(u" ").strip(u'\n').strip() for x in b.find('div', attrs={"class":"part-details"}) if "\n " in x]
        details =  [x.contents[0] for x in b.find('div', attrs={"class":"part-details"}).find_all('h4')]
        detail_dict = {detail:detail_val for detail_val, detail in zip(detail_vals, details)}
        
        #file name
        file_name_dict = {'file_name':i}

        #Permalink
        perma_link = b.find('input', attrs={"type":"text"})['value'].split("/b/")[1]
        
        #create dictionary
        build_dict = {"Permalink": perma_link, "Builder":builder_link}
        #assemble dictionary with all build information
        master_dict = dict(build_dict.items()+builder_dict.items()+build_part_dict.items()+detail_dict.items()+total_price_dict.items()+owner_dict.items()+description_dict.items()+file_name_dict.items())
        
        #append master_dict to build_dict_list
        build_dict_list.append(master_dict)
        
        #destroy bs4 object and close file
        b.decompose()
        a.close()
        
    except:
        pass

In [ ]:
os.chdir(os.path.expanduser('~/Documents/Projects/Data/PCPP/builds/builds_html'))
files = os.listdir(os.getcwd())

build_dict_list = []

for num, x in enumerate(files):
    scrape(x)
    print num,

In [80]:
df = pd.DataFrame(build_dict_list)

In [63]:
os.chdir(os.path.expanduser('~/Documents/Projects/Data/PCPP/builds/')
df.to_csv('all_builds_FINAL.csv', encoding='utf-8', index=False)